In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import geopandas
from shapely.geometry import Point, Polygon
from datetime import datetime

%matplotlib inline

In [2]:
df = pd.read_csv('yellow_tripdata_2016-06.csv')
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,2,2016-06-09 21:06:36,2016-06-09 21:13:08,2,0.79,-73.983360,40.760937,1,N,-73.977463,40.753979,2,6.0,0.5,0.5,0.00,0.0,0.3,7.30
1,2,2016-06-09 21:06:36,2016-06-09 21:35:11,1,5.22,-73.981720,40.736668,1,N,-73.981636,40.670242,1,22.0,0.5,0.5,4.00,0.0,0.3,27.30
2,2,2016-06-09 21:06:36,2016-06-09 21:13:10,1,1.26,-73.994316,40.751072,1,N,-74.004234,40.742168,1,6.5,0.5,0.5,1.56,0.0,0.3,9.36
3,2,2016-06-09 21:06:36,2016-06-09 21:36:10,1,7.39,-73.982361,40.773891,1,N,-73.929466,40.851540,1,26.0,0.5,0.5,1.00,0.0,0.3,28.30
4,2,2016-06-09 21:06:36,2016-06-09 21:23:23,1,3.10,-73.987106,40.733173,1,N,-73.985909,40.766445,1,13.5,0.5,0.5,2.96,0.0,0.3,17.76


In [4]:
# read borough boundaries
polygons = geopandas.read_file('BoroughBoundaries/geo_export_3de3c107-194b-45b3-b929-f8d34b05d8a4.shp')

In [6]:
# create geometry point for geo merge
df['pickup_point'] = df.apply(lambda x: Point(x['pickup_longitude'], x['pickup_latitude']), axis=1)
df['dropoff_point'] = df.apply(lambda x: Point(x['dropoff_longitude'], x['dropoff_latitude']), axis=1)

In [9]:
# Merge trip data with borough boundaries
data_pickup = df[['pickup_point']]
data_pickup.columns = ['geometry']

data_dropoff = df[['dropoff_point']]
data_dropoff.columns = ['geometry']

temp_pick = geopandas.sjoin(geopandas.GeoDataFrame(data_pickup), polygons[['geometry', 'boro_name']],\
                            how="left", op='intersects')
temp_pick.columns = ['geometry', 'index_pick',  'boro_pick']
temp_drop = geopandas.sjoin(geopandas.GeoDataFrame(data_dropoff), polygons[['geometry', 'boro_name']],\
                            how="left", op='intersects')
temp_drop.columns = ['geometry', 'index_drop',  'boro_drop']

data = pd.concat([df, temp_pick[['index_pick',  'boro_pick']], \
                       temp_drop[['index_drop',  'boro_drop']]], axis=1)
# data_boro.drop('geometry', axis=1, inplace=True)
data.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,...,tip_amount,tolls_amount,improvement_surcharge,total_amount,pickup_point,dropoff_point,index_pick,boro_pick,index_drop,boro_drop
0,2,2016-06-09 21:06:36,2016-06-09 21:13:08,2,0.79,-73.983360,40.760937,1,N,-73.977463,...,0.00,0.0,0.3,7.30,POINT (-73.98336029052734 40.76093673706055),POINT (-73.97746276855469 40.75397872924805),3.0,Manhattan,3.0,Manhattan
1,2,2016-06-09 21:06:36,2016-06-09 21:35:11,1,5.22,-73.981720,40.736668,1,N,-73.981636,...,4.00,0.0,0.3,27.30,POINT (-73.98171997070312 40.73666763305664),POINT (-73.98163604736328 40.67024230957031),3.0,Manhattan,4.0,Brooklyn
2,2,2016-06-09 21:06:36,2016-06-09 21:13:10,1,1.26,-73.994316,40.751072,1,N,-74.004234,...,1.56,0.0,0.3,9.36,POINT (-73.99431610107422 40.75107192993164),POINT (-74.00423431396483 40.74216842651367),3.0,Manhattan,3.0,Manhattan
3,2,2016-06-09 21:06:36,2016-06-09 21:36:10,1,7.39,-73.982361,40.773891,1,N,-73.929466,...,1.00,0.0,0.3,28.30,POINT (-73.98236083984375 40.77389144897461),POINT (-73.92946624755859 40.85153961181641),3.0,Manhattan,3.0,Manhattan
4,2,2016-06-09 21:06:36,2016-06-09 21:23:23,1,3.10,-73.987106,40.733173,1,N,-73.985909,...,2.96,0.0,0.3,17.76,POINT (-73.98710632324219 40.73317337036133),POINT (-73.98590850830078 40.7664451599121),3.0,Manhattan,3.0,Manhattan


In [21]:
# Filter the first week in June, 2016
start = datetime.strptime('2016-06-06', '%Y-%m-%d')
end = datetime.strptime('2016-06-13', '%Y-%m-%d')

data['tpep_pickup_datetime'] = pd.to_datetime(data['tpep_pickup_datetime'])
data['tpep_dropoff_datetime'] = pd.to_datetime(data['tpep_dropoff_datetime'])

data = data[(data['tpep_pickup_datetime']  >= start) & (data['tpep_pickup_datetime']  < end)\
           & (data['tpep_dropoff_datetime']  >= start) & (data['tpep_dropoff_datetime']  < end)\
           & (df['tpep_pickup_datetime'] < df['tpep_dropoff_datetime'])]
print('first week selected')


data = data[(data['index_pick'] == 3) & (data['index_drop'] == 3)]
print('manhattan selected')

# data.drop('Unnamed: 0', axis=1, inplace=True)
#df.to_csv('Manhattan201606FirstWeek.csv')

first week selected
manhattan selected


In [22]:
data.shape

(2167850, 25)

In [23]:
# Compute straight line distance
# reference: https://stackoverflow.com/questions/15736995/how-can-i-quickly-estimate-the-distance-between-two-latitude-longitude-points
from math import radians, cos, sin, asin, sqrt
def haversine(lat1, lon1, lat2, lon2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    # Radius of earth in kilometers is 6371
    km = 6371* c
    conv_fac = 0.621371
    
    return km * conv_fac

data['distance_line'] = data.apply(lambda x: haversine(x['pickup_latitude'], x['pickup_longitude'], \
                                        x['dropoff_latitude'], x['dropoff_longitude']), axis=1)

In [24]:
# Compute bearing
# reference: https://gist.github.com/jeromer/2005586
import math
def calculate_initial_compass_bearing(pointA, pointB):
    """
    Calculates the bearing between two points.
    The formulae used is the following:
        θ = atan2(sin(Δlong).cos(lat2),
                  cos(lat1).sin(lat2) − sin(lat1).cos(lat2).cos(Δlong))
    :Parameters:
      - `pointA: The tuple representing the latitude/longitude for the
        first point. Latitude and longitude must be in decimal degrees
      - `pointB: The tuple representing the latitude/longitude for the
        second point. Latitude and longitude must be in decimal degrees
    :Returns:
      The bearing in degrees
    :Returns Type:
      float
    """
    if (type(pointA) != tuple) or (type(pointB) != tuple):
        raise TypeError("Only tuples are supported as arguments")

    lat1 = math.radians(pointA[0])
    lat2 = math.radians(pointB[0])

    diffLong = math.radians(pointB[1] - pointA[1])

    x = math.sin(diffLong) * math.cos(lat2)
    y = math.cos(lat1) * math.sin(lat2) - (math.sin(lat1)
            * math.cos(lat2) * math.cos(diffLong))

    initial_bearing = math.atan2(x, y)

    # Now we have the initial bearing but math.atan2 return values
    # from -180° to + 180° which is not what we want for a compass bearing
    # The solution is to normalize the initial bearing as shown below
    initial_bearing = math.degrees(initial_bearing)
    compass_bearing = (initial_bearing + 360) % 360

    return compass_bearing

data['bearing'] = data.apply(lambda x: calculate_initial_compass_bearing((x['pickup_latitude'], x['pickup_longitude']), \
                                        (x['dropoff_latitude'], x['dropoff_longitude'])), axis=1)

In [25]:
# Sort data by pickup time
df_sorted = data.sort_values(by='tpep_pickup_datetime')
df_sorted['trip_duration'] = df_sorted['tpep_dropoff_datetime'] - df_sorted['tpep_pickup_datetime']

In [33]:
# remove outliers of trip distance and trip duration
min_dist = df_sorted['trip_distance'].quantile(0.1) # 0.6 miles
max_dist = df_sorted['trip_distance'].quantile(0.999) # 10.73 miles, approximately from Upper to Lower Manhattan
dist_filter = (df_sorted['trip_distance'] >= min_dist) & (df_sorted['trip_distance'] <= max_dist)

min_line = df_sorted['distance_line'].quantile(0.1) # 0.473 miles
max_line = df_sorted['distance_line'].max() # 12.97, which is fine
line_filter = (df_sorted['distance_line'] >= min_line)

min_dura = df_sorted['trip_duration'].quantile(0.05) # '0 days 00:02:59'
max_dura = df_sorted['trip_duration'].quantile(0.995) # '0 days 00:49:12'
duration_filter = (df_sorted['trip_duration'] >= min_dura) & (df_sorted['trip_duration'] <= max_dura)

min_count = 1
max_count = 4
passenger_count_filter = (df_sorted['passenger_count'] >= min_count) & (df_sorted['passenger_count'] <= max_count)

df_sorted = df_sorted[dist_filter & line_filter & duration_filter & passenger_count_filter].reset_index()
df_sorted.shape

(1722726, 29)

In [43]:
df_sorted[['index', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
           'passenger_count', 'trip_distance', 'pickup_longitude', 
           'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude', 
           'pickup_point', 'dropoff_point', 'index_pick', 
           'boro_pick', 'index_drop', 'boro_drop',
           'distance_line', 'bearing']].to_csv('Manhattan201606FirstWeek_sorted.csv')